# Finalise facebook comments
Finalises the facebook comments.
And persists it to s3.
See docs for more information.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts
from phoenix.common import utils
from phoenix.tag import export
from phoenix.tag import finalise
from phoenix.tag import object_filters

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parametrise the run execution date.
# Format of the run date
RUN_DATE_FORMAT = "%Y-%m-%d"
# This can be overwritten at execution time by Papermill to enable historic runs and backfills etc.
RUN_DATE = datetime.datetime.today().strftime(RUN_DATE_FORMAT)

# Set Artefacts URL
ARTIFACTS_BASE_URL = f"{artifacts.urls.get_local()}tagging_runs/{MONTH_PREFIX}facebook_posts/"

# OUTPUT
FACEBOOK_COMMENTS_PERSIST = "s3://buildup-dev-us-tables/facebook_comments/parquet_exports/facebook_comments_v1/persisted.parquet"

In [ ]:
# Display params.
print(
ARTIFACTS_BASE_URL,
RUN_DATE,
sep='\n',
)

In [ ]:
# %env DASK_CLUSTER_IP=

In [ ]:
utils.dask_global_init()

In [ ]:
facebook_comments_df = artifacts.dataframes.get(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "facebook_comments_pulled")).dataframe

In [ ]:
facebook_comments_df.head()

In [ ]:
objects = artifacts.dataframes.get(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "object_tensions")).dataframe

In [ ]:
objects.head()

In [ ]:
language_sentiment_objects = artifacts.dataframes.get(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "language_sentiment_objects")).dataframe
language_sentiment_objects.head()

In [ ]:
facebook_comments_final = finalise.join_objects_to_facebook_comments(objects, language_sentiment_objects, facebook_comments_df)

In [ ]:
facebook_comments_final.head()

In [ ]:
facebook_comments_final.dtypes

In [ ]:
_ = artifacts.dataframes.persist(FACEBOOK_COMMENTS_PERSIST, facebook_comments_final)

In [ ]:
_ = artifacts.dataframes.persist(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "facebook_comments_final"), facebook_comments_final)

In [ ]:
artifacts.dataframes.read_schema(artifacts.dataframes.url(ARTIFACTS_BASE_URL, "facebook_comments_final"))